# 데이터 전처리
이 단계에서 사용되는 라이브러리는 TensorFlow Transform이며 이에 대응하는 컴포넌트는 `Transform`
이다. 전처리 과정은 텐서플로 생태계 밖 다양한 도구를 사용할 수 있지만, 텐서플로 연산으로 전처리 과정을
기술하게 되면 이를 그래프 구조로 표현이 가능하게 된다. 그래프 구조로 표현된 전처리 과정은 모델 그래프의
일부로 조립될 수 있단 장점이 생긴다.

동물 분류 모델 서버가 있는데, 이 모델 서버에 연결된 각종 서비스가 제각각 이미지를 전처리하도록 구성한다면
서비스 개발자가 전처리 구현을 빼먹을 수 있다. 이 경우 바르지 못한 추론 결과를 얻게 된다. 이렇게
research 단계와 production 단계가 다른 경우 생기는 오류를 **training-serving skew**라고 한다.

In [26]:
# TFT에는 전처리에 특화된 다양한 연산들이 구비돼 있다.
import os
import tempfile

import apache_beam as beam
import tensorflow as tf
import tensorflow_transform.beam.impl as tft_beam
import tensorflow_transform as tft
import tfx.proto.transform_pb2
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils
from tfx.components import (ImportExampleGen, StatisticsGen, SchemaGen,
                            ExampleValidator, Transform)
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2, transform_pb2

## TFT 라이브러리로 독립환경 데이터 전처리

In [2]:
# TFT의 핵심 함수. 모든 전처리 단계는 여기에 텐서플로 연산으로 기술돼야 한다.
# 출력은 피처명을 키로, 변환된 피처를 값으로 하는 dict 형식이어야 한다. 이 출력 피처명이 그래프 상
# 입력 노드의 이름이 될 것이다.
def preprocessing_fn(inputs):
    x = inputs['x']
    x_normalized = tft.scale_to_0_1(x)
    return {
        'x_xf': x_normalized
    }

In [3]:
# 전형적인 이미지 처리 과정: 이미 정의된 함수가 있다면 데코레이터를 써서 바로 전처리 함수로 만들 수도.
def process_image(raw_image, width=300, height=300):
    img_rgb = tf.io.decode_jpeg(raw_image, channels=3)
    img_gray = tf.image.rgb_to_grayscale(img_rgb)
    img = tf.image.convert_image_dtype(img_gray, tf.float32)
    img_resized = tf.image.resize_with_pad(
        img, target_height=height, target_width=width
    )
    return tf.reshape(img_resized, [-1, height, width, 1])


def preprocessing_decorator(fn):
    def wrapper(*args, **kwargs):
        return fn(*args, **kwargs)

    return wrapper

In [37]:
img_path = 'data/animals/images/dog.jpg'
raw_image = tf.io.read_file(img_path)
pseudo_preprocessing_fn = preprocessing_decorator(process_image)
image_processed = pseudo_preprocessing_fn(raw_image, width=512, height=512)

In [39]:
raw_data = [
    {"x": 1.11},
    {"x": 4.15},
    {"x": 99.89},
]

In [54]:
from collections import defaultdict

with tf.io.TFRecordWriter("data/x.tfrecord") as writer:
    feature = defaultdict(list)
    for sample in raw_data:
        for key, val in sample.items():
            feature[key].append(val)

    features = {}
    for name, vals in feature.items():
        features[key] = tf.train.Feature(
            float_list=tf.train.FloatList(value=vals)
        )
    print(features)
    example = tf.train.Example(
        features=tf.train.Features(feature=features)
    )
    writer.write(example.SerializeToString())

{'x': float_list {
  value: 1.1100000143051147
  value: 4.150000095367432
  value: 99.88999938964844
}
}


In [42]:
# 간단한 데이터셋이므로 직접 스키마를 생성한다.
raw_data_metadata = dataset_metadata.DatasetMetadata(
    schema_utils.schema_from_feature_spec(
        # `raw_data` 각 x값은 단일 float32 타입이다.
        {"x": tf.io.FixedLenFeature([], tf.float32)}
    )
)

In [55]:
with beam.Pipeline() as pipeline:
    with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
        # tfrecord_file = "data/x.tfrecord"
        # collection = (
        #         pipeline | beam.io.ReadFromTFRecord(tfrecord_file)
        # )

        transformed_dataset, transform_fn = (
            # TODO(me): 읽은 tfrecord 파일을 넘기도록 해볼 것.
                (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(preprocessing_fn)
        )

transformed_data, transformed_metadata = transformed_dataset

INFO:tensorflow:Assets written to: /tmp/tmp11l4jabm/tftransform_tmp/e805bd1a861140f8a83a1cebc384cb7a/assets


INFO:tensorflow:Assets written to: /tmp/tmp11l4jabm/tftransform_tmp/e805bd1a861140f8a83a1cebc384cb7a/assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


In [61]:
transformed_data

[{'x_xf': 0.0}, {'x_xf': 0.030775461}, {'x_xf': 1.0}]

In [62]:
transformed_metadata  # 변환된 스킴 내용을 볼 수 있다.

BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "x_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
, '_output_record_batches': False}, deferred_metadata=[{'_schema': feature {
  name: "x_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
, '_output_record_batches': True}], asset_map={})

In [63]:
transform_fn

(['/tmp/tmp11l4jabm/tftransform_tmp/172db3b02d0f49e2bf96bdc746cf17f2'],
 BeamDatasetMetadata(dataset_metadata={'_schema': feature {
   name: "x_xf"
   type: FLOAT
   presence {
     min_fraction: 1.0
   }
   shape {
   }
 }
 , '_output_record_batches': False}, deferred_metadata=[{'_schema': feature {
   name: "x_xf"
   type: FLOAT
   presence {
     min_fraction: 1.0
   }
   shape {
   }
 }
 , '_output_record_batches': True}], asset_map={}))

## 파이프라인 일부로 전처리 단계 통합

### 전처리를 위한 준비 단계

In [67]:
# bucketized column의 의미:
age = tf.feature_column.feature_column.numeric_column("age")
age_buckets = tf.feature_column.feature_column.bucketized_column(
    age,
    boundaries=[18, 25, 30])  # buckets: (, 18), [18, 25), [25, 30)

In [2]:
# 피처명, 차원(가능한 값의 수)
ONE_HOT_FEATURES = {
    "product": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90,
}

# 피처명, 버킷 수(범위 수)
BUCKET_FEATURES = {
    "zip_code": 10
}

# 피처명, 미정의
TEXT_FEATURES = {
    "consumer_complaint_narrative": None
}

LABEL_KEY = "consumer_disputed"

In [3]:
# 헬퍼 함수들 - `preprocessing_fn`과 그에 필요한 함수들로 Transform에게 넘길 수 있도록 한
# 파이썬 모듈 파일로 작성해 놓아야 한다.
def transform_name(key: str):
    return key + "_xf"


def fill_in_missing(x):
    if isinstance(x, tf.sparse.SparseTensor):
        default_value = "" if x.dtype == tf.string else 0
        dense_shape = [x.dense_shape[0], 1]
        x = tf.sparse.to_dense(
            # `tf.sparse.SparseTensor`의 별칭.
            tf.SparseTensor(indices=x.indices,  # non-0 위치.
                            values=x.values,  # `indices`에 지정된 위치에 들어갈 값들.
                            dense_shape=dense_shape),
            default_value=default_value
        )
    return tf.squeeze(x, axis=1)  # 크기 1인 차원을 없앤다. `axis`로 축소할 차원 나열.


def convert_num_to_one_hot(label_tensor, num_labels=2):
    one_hot_tensor = tf.one_hot(
        indices=label_tensor,
        depth=num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])  # not-fixed-size batch


def convert_zip_code(zip_code):
    if zip_code == "":
        zip_code = "00000"

    zip_code = tf.strings.regex_replace(zip_code, r"X{0,5}", "0")
    zip_code = tf.strings.to_number(zip_code, out_type=tf.int32)
    return zip_code


# TFT에서 가장 중요한 함수!
def preprocessing_fn(inputs):
    outputs = {}

    for key in ONE_HOT_FEATURES.keys():
        dim = ONE_HOT_FEATURES[key]
        idx = tft.compute_and_apply_vocabulary(
            fill_in_missing(inputs[key]), top_k=dim + 1
        )
        outputs[transform_name(key)] = convert_num_to_one_hot(
            idx, num_labels=dim + 1
        )

    for key, bucket in BUCKET_FEATURES.items():
        temp_feature = tft.bucketize(
            fill_in_missing(inputs[key]),  # TODO(me): `convert_zip_code` 안에 있던 객체였음.
            bucket,
            # always_return_num_quantiles=False
        )

        outputs[transform_name(key)] = convert_num_to_one_hot(
            temp_feature, num_labels=bucket + 1
        )

        for key in TEXT_FEATURES.keys():
            outputs[transform_name(key)] = fill_in_missing(inputs[key])

        outputs[transform_name(LABEL_KEY)] = fill_in_missing(inputs[LABEL_KEY])

    return outputs

### 이전 단계를 수행할 컴포넌트 호출
1. ExampleGen: 파이프라인 진입점. 데이터 소스를 파이프라인에서 소비할 아티팩트로 변환.
2. StatisticsGen: 피처별 요약 통계 생성.
3. SchemaGen: stats 아티팩트로 추론한 스키마.
4. ExampleValidator(이번에는 생략): 스키마를 기준으로 집계된 통계에서 이상치 검출.
5. **Transform**: `preprocessing_fn`으로 기술한 전처리 단계를 실행하여 전처리된 데이터와
        전처리 그래프 함수를 아티팩트로 반환.

In [3]:
context = InteractiveContext()

In [8]:
# 1. Data collection
split_dir = os.path.join("data", "complaints", "splits")

input_config = example_gen_pb2.Input(
    splits=[
        example_gen_pb2.Input.Split(name="train", pattern="train.tfrecord"),
        example_gen_pb2.Input.Split(name="eval", pattern="eval.tfrecord"),
        example_gen_pb2.Input.Split(name="test", pattern="test.tfrecord")])

example_gen = ImportExampleGen(
    input_base=os.path.join(split_dir),
    input_config=input_config)
context.run(example_gen)
example_artifact = example_gen.outputs["examples"]

In [14]:
example_artifact.get()

[Artifact(artifact: id: 4
 type_id: 14
 uri: "/tmp/tfx-interactive-2023-01-13T08_18_46.112624-up0c6laf/ImportExampleGen/examples/4"
 properties {
   key: "split_names"
   value {
     string_value: "[\"train\", \"eval\", \"test\"]"
   }
 }
 custom_properties {
   key: "file_format"
   value {
     string_value: "tfrecords_gzip"
   }
 }
 custom_properties {
   key: "input_fingerprint"
   value {
     string_value: "split:train,num_files:1,total_bytes:50284526,xor_checksum:1673597782,sum_checksum:1673597782\nsplit:eval,num_files:1,total_bytes:19880832,xor_checksum:1673597786,sum_checksum:1673597786\nsplit:test,num_files:1,total_bytes:30805677,xor_checksum:1673597793,sum_checksum:1673597793"
   }
 }
 custom_properties {
   key: "payload_format"
   value {
     string_value: "FORMAT_TF_EXAMPLE"
   }
 }
 custom_properties {
   key: "span"
   value {
     int_value: 0
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "

In [15]:
# 2. stats
statistics_gen = StatisticsGen(examples=example_artifact)
context.run(statistics_gen)
statistics_artifact = statistics_gen.outputs["statistics"]

In [16]:
context.show(statistics_artifact)

In [17]:
# 3. schema
schema_gen = SchemaGen(statistics=statistics_artifact, infer_feature_shape=True)
context.run(schema_gen)
schema_artifact = schema_gen.outputs["schema"]

In [19]:
context.show(schema_artifact)

,Type,Presence,Valency,Domain
Feature name,,,,
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'consumer_complaint_narrative',BYTES,required,,-
'consumer_disputed',FLOAT,required,,-
'issue',STRING,required,,'issue'
'product',STRING,required,,'product'
'state',STRING,required,,'state'
'sub_issue',STRING,required,,'sub_issue'
'sub_product',STRING,required,,'sub_product'


,Values
Domain,
'company_response',"'Company believes complaint caused principally by actions of third party outside the control or direction of the company', 'Company believes complaint is the result of an isolated error', 'Company believes complaint relates to a discontinued policy or procedure', 'Company believes complaint represents an opportunity for improvement to better serve consumers', 'Company believes it acted appropriately as authorized by contract or law', 'Company believes the complaint is the result of a misunderstanding', 'Company can\'t verify or dispute the facts in the complaint', 'Company chooses not to provide a public response', 'Company disputes the facts presented in the complaint', 'Company has responded to the consumer and the CFPB and chooses not to provide a public response', 'nan'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect exchange rate', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender damaged or destroyed vehicle', 'Lender repossessed or sold the vehicle', 'Lender sold the property', 'Loan modification,collection,foreclosure', 'Loan servicing, payments, escrow account', 'Lost or stolen check', 'Lost or stolen money order', 'Making/receiving payments, sending money', 'Managing the line of credit', 'Managing the loan or lease', 'Managing, opening, or closing account', 'Money was not available when promised', 'Other', 'Other fee', 'Other service issues', 'Other transaction issues', 'Overdraft, savings or rewards features', 'Overlimit fee', 'Payment to acct not credited', 'Payoff process', 'Privacy', 'Problems caused by my funds being low', 'Problems when you are unable to pay', 'Received a loan I didn\'t apply for', 'Rewards', 'Sale of account', 'Settlement process and costs', 'Shopping for a line of credit', 'Shopping for a loan or lease', 'Taking out the loan or lease', 'Taking/threatening an illegal action', 'Transaction issue', 'Unable to get credit report/credit score', 'Unauthorized transactions/trans. issues', 'Unexpected/Other fees', 'Unsolicited issuance of credit card', 'Using a debit or ATM card', 'Wrong amount charged or received'"
'product',"'Bank account or service', 'Consumer Loan', 'Credit card', 'Credit reporting', 'Debt collection', 'Money transfers', 'Mortgage', 'Other financial service', 'Payday loan', 'Prepaid card', 'Student loan', 'Virtual currency'"
'state',"'AE', 'AK', 'AL', 'AP', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FM', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS

In [20]:
# 4. validation
example_validator = ExampleValidator(statistics=statistics_artifact,
                                     schema=schema_artifact)
context.run(example_validator)
anomalies_artifact = example_validator.outputs['anomalies']

In [21]:
context.show(anomalies_artifact)

In [27]:
# 5. preprocessing
module_file = 'transform_module.py'
transform = Transform(
    examples=example_artifact,
    schema=schema_artifact,
    module_file=module_file,  # 파일명은 자유이나, `preprocessing_fn`이 꼭 포함돼야 한다.
    splits_config=transform_pb2.SplitsConfig(analyze=["train"], transform=["eval"])
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform_module.py -> build/lib
copying trainer_module.py -> build/lib
copying hello.py -> build/lib
installing to /tmp/tmpoccgbe10
running install
running install_lib
copying build/lib/transform_module.py -> /tmp/tmpoccgbe10
copying build/lib/trainer_module.py -> /tmp/tmpoccgbe10
copying build/lib/hello.py -> /tmp/tmpoccgbe10
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpoccgbe10/tfx_user_code_Trans

/home/hakjun/projects/pipeline/venv/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing /tmp/tfx-interactive-2023-01-13T08_18_46.112624-up0c6laf/_wheels/tfx_user_code_Transform-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f-py3-none-any.whl
Processing /tmp/tfx-interactive-2023-01-13T08_18_46.112624-up0c6laf/_wheels/tfx_user_code_Transform-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f-py3-none-any.whl
Processing /tmp/tfx-interactive-2023-01-13T08_18_46.112624-up0c6laf/_wheels/tfx_user_code_Transform-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f-py3-none-any.whl


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2023-01-13T08_18_46.112624-up0c6laf/Transform/transform_graph/9/.temp_path/tftransform_tmp/751cd8e613614e1fb34ee444ae1dce94/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2023-01-13T08_18_46.112624-up0c6laf/Transform/transform_graph/9/.temp_path/tftransform_tmp/751cd8e613614e1fb34ee444ae1dce94/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 9
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [28]:
# transform_graph, transformed_examples 다음 Trainer에게 필요한 아티팩트들.
transform.outputs.keys()

dict_keys(['transform_graph', 'transformed_examples', 'updated_analyzer_cache', 'pre_transform_schema', 'pre_transform_stats', 'post_transform_schema', 'post_transform_stats', 'post_transform_anomalies'])

In [30]:
transform.outputs.keys()

dict_keys(['transform_graph', 'transformed_examples', 'updated_analyzer_cache', 'pre_transform_schema', 'pre_transform_stats', 'post_transform_schema', 'post_transform_stats', 'post_transform_anomalies'])